In [41]:
# Imports
import pandas as pd
import folium
import folium.plugins as plugins
import ast
print("finished imports")

finished imports


# Dunker Class

In [42]:
class Dunker:
    def __init__(self, name, location, coord, league, link, avatar="default_avi.jpeg"):
        self.name = name
        self.location = location
        self.coordinates = ast.literal_eval(coord)
        self.league = league
        self.link = link
        self.avatar = avatar
    def getCoordinates(self):
        return self.coordinates
#     def __str__(self):
#         return f"Name: {self.name},\nLocation: {self.location}\nLeague: {self.league}"
    def get_link(self):
        return self.link
    def __repr__(self):
        # This is how I want dunker info to be displayed to the end user.
        return f"<img src={self.avatar} alt='avatar' height='50px' width='50px'></img>{self.name}\nLocation: {self.location}\n <a href={self.link} target=_blank>Insta</a>" 
    def __eq__(self, other):
        return self.coordinates == other.coordinates

In [43]:
dunkerdf = pd.read_csv("dunker_data.csv")
dunkerdf.loc[dunkerdf.League=="PRO"]
print(len(dunkerdf))

181


In [44]:
# Create list of Dunker objects by looping through dunkerdf

dunker_list = []
for dunker in dunkerdf.itertuples():
    name, loc, coord, league, link = dunker[1::]
    dunker_list.append(Dunker(name, loc, coord, league, link))

In [45]:
for d in dunker_list[::10]:
    print(d.get_link())

https://instagram.com/demar_derozan
https://instagram.com/javalemcgee
https://instagram.com/barryathree
https://instagram.com/steve_franchise3
https://instagram.com/therobertpackorganization
nan
https://instagram.com/jclarkthejumper
https://instagram.com/remcofrankin
https://instagram.com/jordankilganon
https://instagram.com/miller_dunks
https://instagram.com/brodiestephens_
https://instagram.com/monikbarsuk
https://instagram.com/Ez_dondon7
https://instagram.com/jumpmau
https://instagram.com/Mikhrabczuk
https://instagram.com/flysteezo
https://instagram.com/Trax_basket
https://instagram.com/twist_dunks
https://instagram.com/kadour_ziani_official


### Many of these Dunkers have identical coordinates. They require ClusterMarkers as opposed to Clusters. From dunkerdf, I can grab each coordinate's number of occurences

In [46]:
coord_counts = dunkerdf["Coordinates"].value_counts()
coord_counts

(37.0902, -95.7129)                 72
(48.856614, 2.3522219)               6
(34.0522342, -118.2436849)           4
(47.8388, 35.139567)                 3
(45.4041718, -71.8929107)            3
                                    ..
(35.0059914, -77.2210654)            1
(32.8323225, -79.82842579999999)     1
(49.321374, 6.118281899999999)       1
(42.3149367, -83.03636329999999)     1
(48.637692, 4.947418)                1
Name: Coordinates, Length: 86, dtype: int64

### I want a list of just coordinates that appear more than once.

In [47]:
# I can use the where() method to change the series to meet a certain condition (value > 1)
coord_counts = coord_counts.where(coord_counts > 1)
coord_counts

(37.0902, -95.7129)                 72.0
(48.856614, 2.3522219)               6.0
(34.0522342, -118.2436849)           4.0
(47.8388, 35.139567)                 3.0
(45.4041718, -71.8929107)            3.0
                                    ... 
(35.0059914, -77.2210654)            NaN
(32.8323225, -79.82842579999999)     NaN
(49.321374, 6.118281899999999)       NaN
(42.3149367, -83.03636329999999)     NaN
(48.637692, 4.947418)                NaN
Name: Coordinates, Length: 86, dtype: float64

### Great. But now there's a bunch of N/A values and the list is still too long.

In [48]:
# Drop N/A values and convert pd.Series to a list
coord_list = coord_counts.dropna()
coord_list = coord_list.astype(int)
coord_list

(37.0902, -95.7129)           72
(48.856614, 2.3522219)         6
(34.0522342, -118.2436849)     4
(47.8388, 35.139567)           3
(45.4041718, -71.8929107)      3
(51.919438, 19.145136)         3
(48.379433, 31.1655799)        2
(38.7374943, -104.7888615)     2
(35.86166, 104.195397)         2
(55.169438, 23.881275)         2
(45.4642035, 9.189982)         2
(-31.9523123, 115.861309)      2
(40.6781784, -73.9441579)      2
(51.5073509, -0.1277583)       2
(-37.8136276, 144.9630576)     2
(46.227638, 2.213749)          2
Name: Coordinates, dtype: int32

### Awesome! Let's design a funtion that takes two parameters (list of coords and list of dunker objects) and places markers on the map

In [49]:
def to_map(dlist, clist):
    # dlist: List of dunker objects 
    # clist: list of coordinates that appear more than once in dunkerdf
    newclist = clist.astype(int)
    for c in newclist.index:
        # for every coordinate in clist, make a MarkerCluster. The locations are a list of current coordinate * number of occurences)
        locations = [ast.literal_eval(c) for _ in range(newclist[c])]
        names = list(filter(lambda d: d.coordinates == ast.literal_eval(c), dlist))
        plugins.MarkerCluster(locations=locations, popups=names, icon=folium.Icon(color=["red" for _ in range(newclist[c])])).add_to(m)
    for d in dlist:
        if str(d.coordinates) not in newclist.index:
            folium.Marker(location=d.coordinates, popup=d.__str__(), icon=color_by_league(d)).add_to(m)

In [50]:
map1 = folium.Map(zoom_start=50, control_scale=True)

In [51]:
def same_coord(d: Dunker, cl: list):
    #dl: dunker
    #cl: coordinate_list
    for i, c in enumerate(cl):
        if str(d.coordinates) == c:
            # Returns index of coordinate list with coordinates that match the dunkers
            return i

In [52]:
def icon_by_league(d: Dunker):
    if d.league == "PRO":
        return folium.Icon(color="red", icon="male", prefix="fa")
    if d.league == "AMATEUR":
        return folium.Icon(color="green", icon="male", prefix="fa")
    if d.league == "NBA":
        return folium.Icon(color="black", icon="male", prefix="fa")
    if d.league == "FORMER NBA":
        return folium.Icon(color="darkpurple", icon="male", prefix="fa")
    if d.league == "FORMER":
        return folium.Icon(color="cadetblue", icon="male", prefix="fa")

In [53]:
# Let's create a Marker Cluster for every coordinate that needs one
cluster_list = []
cint = coord_list.index
for coordinate in cint:
    cluster_list.append(plugins.MarkerCluster())
cluster_list
i=0
a=0
for d in dunker_list:
    
    if str(d.coordinates) in cint:
        i+=1
        # add to cluster with those coordinates
        folium.Marker(location=d.coordinates, 
                      popup=d,
                      icon=icon_by_league(d)
                     ).add_to(cluster_list[same_coord(d, cint)])
    else:
        a+=1
        # add regular marker to the map
        folium.Marker(location=d.coordinates,
                      popup=d, icon=icon_by_league(d)
                     ).add_to(map1)
# Now let's loop through dunkers and if their coordinates match markerCluster coordinates, add to marker cluster
# add regular marker to the map
print(f"cluster dunkers: {i}, reg dunkers: {a}")

# Loop over list of MarkerClusters and add them to the folium map
for mc in cluster_list:
    mc.add_to(map1)


cluster dunkers: 111, reg dunkers: 70


In [54]:
map1._repr_html_()
map1.save("thismap.html")
